In [21]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler #scikit-learn==0.21.3
from xgboost import XGBClassifier #xgboost==0.90
from sklearn.manifold import TSNE
import xgboost
from sklearn.preprocessing import LabelEncoder

In [22]:
xgboost.__version__

'0.90'

## G1 predictions

In [23]:
data = pd.read_csv('../Field_training_data_justMMETSP.csv')
labels = pd.read_csv('../Field_training_labels_justMMETSP.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams.csv')

G1_TPM_merged_lats = pd.read_csv('../../g1Surface/G1_surface_tpm_updatedMarferret_marmicroDb2023_noOutliers_fixedTPM_fall2023.csv')

In [24]:
G1_TPM_merged_lats.columns[(G1_TPM_merged_lats == 0).all()]

Index(['PF14259'], dtype='object')

In [27]:
features = features['Pfam'].str.split('.').str[0]

In [28]:
len(features)

1046

In [29]:
original_features = set(G1_TPM_merged_lats.columns)

In [30]:
missing_features = set(features) - original_features
missing_features

{'PF02577',
 'PF03986',
 'PF04413',
 'PF08017',
 'PF08217',
 'PF08802',
 'PF10102',
 'PF13427'}

In [31]:
#Add the missing features to G1_TPM_merged_lats, filled with 0 values
for feature in missing_features:
    G1_TPM_merged_lats[feature] = 0

In [32]:
G1_TPM_merged_lats = G1_TPM_merged_lats[features]

In [34]:
train_data = train_data[features]

In [35]:
train_data

,PF00003,PF00004,PF00005,PF00006,PF00009,PF00011,PF00013,PF00017,PF00022,PF00026,...,PF16656,PF16670,PF16861,PF16864,PF16867,PF16874,PF16880,PF16940,PF16987,PF17070
0,0.00000,2445.244315,3511.070480,2092.01536,3084.234790,5283.95143,499.315640,0.0000,785.471090,274.16400,...,23.04660,0.000,5.25132,0.0,19.1588,0.0,25.58290,0.000000,5.24855,0.0000
1,0.00000,3020.821700,2927.936428,1767.59884,5531.772976,4466.12698,585.088180,0.0000,855.852670,406.81500,...,0.00000,0.000,5.87712,0.0,16.9038,0.0,22.38250,0.000000,0.00000,0.0000
2,49.26959,1144.960489,1277.601550,768.94586,1366.186920,3697.23980,311.739820,0.0000,2014.111170,285.84694,...,3.55636,0.000,0.00000,0.0,0.0000,0.0,15.98250,60.403976,0.00000,0.0000
3,0.00000,735.994072,2160.446631,140.30273,1795.961440,74.63347,311.482154,0.0000,150.774890,150.36376,...,14.12888,0.000,0.00000,0.0,0.0000,0.0,75.22080,43.159340,0.00000,0.0000
4,26.39806,2391.571913,1756.057402,3248.09140,6595.681330,1062.65200,190.391680,0.0000,1872.674065,381.40553,...,0.00000,0.000,28.95807,0.0,0.0000,0.0,20.71440,16.672800,0.00000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648,0.00000,848.648107,939.931050,610.16478,2605.628840,0.00000,505.655021,0.0000,46.897830,12.96344,...,0.00000,0.000,0.00000,0.0,0.0000,0.0,12.07640,17.349600,0.00000,0.0000
651,0.00000,362.558379,312.456080,109.71066,216.776890,76.02075,81.750730,0.0000,177.815230,25.71119,...,0.00000,0.000,0.00000,0.0,0.0000,0.0,16.44190,4.577090,0.00000,0.0000
652,0.00000,596.994332,667.995930,174.25081,53.931910,401.58490,201.488640,0.0000,227.160990,249.82880,...,0.00000,0.000,0.00000,0.0,0.0000,0.0,0.00000,0.000000,0.00000,0.0000
653,0.00000,2689.748840,1577.127220,644.04063,3474.856100,1328.19190,266.495400,0.0000,236.883390,647.61326,...,25.49890,36.506,0.00000,0.0,0.0000,0.0,20.50850,74.253000,0.00000,31.4703


In [36]:
X,y = train_data, train_labels['Trophic mode']
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [37]:
#gamma: 0.5, learning_rate: 0.2, max_depth: 3, n_estimators: 100, reg_lambda: 0.0 
model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.0,
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=0.0, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [38]:
# Initialize empty DataFrames to store results
input_pfams = pd.DataFrame()
output_predictions = pd.DataFrame()
excluded_pfams = pd.DataFrame()

In [39]:
for i in range (0, 30):
    features1 = features.sample(frac =.9)
    excluded = list(set(features) - set(features1))
    G1 = G1_TPM_merged_lats.copy()
    G1[excluded] = 0
    G1 = scaler.transform(G1)
    xg_predictions = model.predict(G1)
    input_pfams[f'col{i}'] = features1
    output_predictions[f'col{i}'] = xg_predictions
    excluded_pfams[f'col{i}'] = excluded

In [40]:
input_pfams.to_csv('/Users/elainathomas/Dropbox/grad/research/g1Surface/bootstrapPfams_lambert.csv')

In [41]:
output_predictions.to_csv('/Users/elainathomas/Dropbox/grad/research/g1Surface/bootstrapPredictions_lambert.csv')

In [42]:
excluded_pfams.to_csv('/Users/elainathomas/Dropbox/grad/research/g1Surface/bootstrapPfamsExcluded_lambert.csv')